In [1]:
import numpy as np

In [2]:
import numpy as np
from dafnedset import base_simple as bs

In [3]:
opts = bs.FunBufferOptions(batch_size=30,niter=2)
buf = bs.FunBuffer(options=opts, providers=[[np.ones(13)*i for i in range(14)]])

In [4]:
opts = bs.FunBufferOptions(batch_size=30,niter=2)
buf2 = bs.FunBuffer(options=opts, providers=[[np.ones(13)*i for i in range(14)]])

In [5]:
buf + buf2

In [6]:
buf = buf + buf2

In [7]:
a = bs.fill_streams(buf)

In [8]:
a

[<method-wrapper '__next__' of list_iterator object at 0x7fe6f780c2e0>,
 <method-wrapper '__next__' of list_iterator object at 0x7fe6f780c340>]

In [9]:
buf.buffer

In [10]:
bs.advance(buf)

(None, [])

In [11]:
try:
    buf.fill_buffer()
except StopIteration:
    test = True
assert test is True

In [12]:
buf.fill_streams()

In [13]:
buf.streams

[<method-wrapper '__next__' of list_iterator object at 0x7fe6f77f2c70>,
 <method-wrapper '__next__' of list_iterator object at 0x7fe6f780c460>]

In [14]:
buf.fill_buffer()

In [15]:
buf.buffer

[array([0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.]),
 array([0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.])]

In [16]:
buf.iteration

[1, 1]

In [17]:
bbb = bs.fill_streams(buf)

In [18]:
bbb

[None, None]

In [19]:
buf.streams

[<method-wrapper '__next__' of list_iterator object at 0x7fe6f77f2c70>,
 <method-wrapper '__next__' of list_iterator object at 0x7fe6f780c460>]

In [20]:
bs.combine_streams(buf.streams,bbb)

([<method-wrapper '__next__' of list_iterator object at 0x7fe6f77f2c70>,
  <method-wrapper '__next__' of list_iterator object at 0x7fe6f780c460>],
 [0, 0])

In [21]:
tot=0

In [22]:
sasa = next(buf)
print(sasa)
print(len(sasa))
print(buf.buffer)
tot = tot+len(sasa)

[0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
 0. 0. 1. 1. 1. 1.]
30
[array([1., 1., 1., 1., 1., 1., 1., 1., 1.]), array([1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1.])]


In [23]:
tot/13/2/2

0.5769230769230769

In [24]:
buf.buffer

[array([1., 1., 1., 1., 1., 1., 1., 1., 1.]),
 array([1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1.])]

In [25]:
buf.cache

[array([0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
        0., 0., 0., 0., 0., 0., 0., 0., 0., 1., 1., 1., 1.])]

In [26]:
from dafnedset import datasets_simple as ds

In [27]:
from dafnedset import fun_ops as fop

In [28]:
from importlib import reload

In [29]:
ds = reload(ds)

In [30]:
ds.CASES

{'size': 'SELECT max(iid) FROM indice_c;',
 'query': "\n                      SELECT a.estacion, a.sid, s.time, a.tiempo, a.norte, a.este, a.altura\n                      FROM\n                        (SELECT max(estacion) estacion, max(sid) sid,\n                                array_agg(to_date(i.yymmmdd,'YYMONDD')\n                                    ORDER BY to_date(i.yymmmdd,'YYMONDD') ) tiempo,\n                                array_agg(north::real ORDER BY to_date(i.yymmmdd,'YYMONDD') ) norte,\n                                array_agg(east::real ORDER BY to_date(i.yymmmdd,'YYMONDD') ) este,\n                                array_agg(up::real ORDER BY to_date(i.yymmmdd,'YYMONDD') ) altura\n                         FROM indice_c i\n                         LEFT JOIN http_tseries t USING (estacion, yymmmdd)\n                         GROUP BY iid HAVING iid <@ int8range(%s,%s) ) AS a\n                      JOIN usgs_sismos s ON a.sid = s.ogc_fid;\n                      ",
 'columns

In [31]:
cfb = ds.DefaultQuerys.positive()

In [32]:
ffb = ds.DefaultQuerys.fake(lambda: np.random.rand(61*3).reshape(3,61).tolist(),1000,100)

In [33]:
ffb.write_parquet('synth.pq')

In [34]:
ffb2 = fop.read_parquet('synth.pq')

In [35]:
for i in ffb2:
    batch = i
    print(f'{len(batch)}',end='...')

100...100...100...100...100...100...100...100...100...100...

In [36]:
cfb = fop.read_db(ds.CASES,batch_size=123)

In [37]:
cfb

In [38]:
for i in cfb:
    batch = i
    print(f'{len(batch)}',end='...')

123...123...123...123...123...123...123...123...123...123...123...123...123...123...123...123...123...123...123...123...123...123...123...123...123...123...123...123...123...123...123...123...123...123...123...123...123...123...123...123...123...123...123...123...123...123...123...123...123...123...123...123...123...123...123...123...123...123...123...123...123...123...123...123...123...123...123...123...123...123...123...123...123...123...123...123...123...123...123...36...

In [39]:
ffb.write_parquet

<bound method DefaultQuerys.write_parquet of <dafnedset.datasets_simple.DefaultQuerys object at 0x7fe6b266fdf0>>

In [40]:
batch.to_pandas()

,est,iid,ep,t,norte,este,altura
0,J607,5164,1249815355110,"[2009-07-10, 2009-07-11, 2009-07-12, 2009-07-1...","[0.222077, 0.225192, 0.225223, 0.223621, 0.223...","[0.278051, 0.273104, 0.275516, 0.277374, 0.277...","[0.657112, 0.667575, 0.660301, 0.658336, 0.674..."
1,RGON,1432,1479034976340,"[2016-10-14, 2016-10-15, 2016-10-16, 2016-10-1...","[-0.2208, -0.2211, -0.220961, -0.219686, -0.22...","[0.785774, 0.786831, 0.785973, 0.787382, 0.788...","[0.185904, 0.174701, 0.177987, 0.172304, 0.173..."
2,PUCA,1353,1482675747010,"[2016-11-25, 2016-11-26, 2016-11-27, 2016-11-2...","[-0.777772, -0.778052, -0.777138, -0.777206, -...","[-0.495856, -0.498015, -0.498214, -0.496385, -...","[0.775837, 0.775968, 0.773589, 0.774267, 0.773..."
3,YAIG,2590,1397831244920,"[2014-03-19, 2014-03-20, 2014-03-21, 2014-03-2...","[0.913235, 0.914604, 0.913449, 0.912496, 0.911...","[0.03689, 0.039529, 0.036231, 0.038023, 0.0414...","[-0.033714, -0.02567, -0.027661, -0.02081, -0...."
4,J850,4238,1299824750300,"[2011-02-09, 2011-02-10, 2011-02-11, 2011-02-1...","[0.091741, 0.089267, 0.090665, 0.091202, 0.089...","[-0.504517, -0.505197, -0.50523, -0.502533, -0...","[0.287978, 0.284853, 0.287995, 0.285578, 0.284..."
5,J996,4258,1299822384120,"[2011-02-09, 2011-02-10, 2011-02-11, 2011-02-1...","[0.704569, 0.704885, 0.705825, 0.705525, 0.705...","[-0.742102, -0.743513, -0.742308, -0.743233, -...","[0.130195, 0.128415, 0.131443, 0.127294, 0.127..."
6,RGHD,1432,1479034976340,"[2016-10-14, 2016-10-15, 2016-10-16, 2016-10-1...","[-0.738343, -0.736548, -0.738469, -0.737071, -...","[-0.104728, -0.104464, -0.106085, -0.106067, -...","[0.478351, 0.475037, 0.480516, 0.474122, 0.470..."
7,TEMA,1432,1479034976340,"[2016-10-14, 2016-10-15, 2016-10-16, 2016-10-1...","[-0.564836, -0.563371, -0.565403, -0.566212, -...","[-0.611245, -0.61149, -0.610658, -0.609058, -0...","[0.175698, 0.166423, 0.169687, 0.169262, 0.165..."
8,J159,4258,1299822384120,"[2011-02-09, 2011-02-10, 2011-02-11, 2011-02-1...","[0.374545, 0.374794, 0.374358, 0.374383, 0.375...","[-0.835438, -0.836291, -0.837153, -0.835695, -...","[0.523939, 0.520942, 0.52704, 0.521701, 0.5230..."
9,J452,1697,1460737506220,"[2016-03-16, 2016-03-17, 2016-03-18, 2016-03-1...","[0.620439, 0.620608, 0.620735, 0.618675, 0.621...","[0.934742, 0.934593, 0.933079, 0.933278, 0.932...","[0.577702, 0.577467, 0.577937, 0.574776, 0.574..."


In [41]:
fop.write_parquet(fop.read_db(ds.CASES),'casos.pq')

In [42]:
l=0
for i in fop.read_parquet('casos.pq'):
    batch = i
    l += len(batch)
    print(f'{len(batch)}, accum {l}',end='...')

100, accum 100...100, accum 200...100, accum 300...100, accum 400...100, accum 500...100, accum 600...100, accum 700...100, accum 800...100, accum 900...100, accum 1000...100, accum 1100...100, accum 1200...100, accum 1300...100, accum 1400...100, accum 1500...100, accum 1600...100, accum 1700...100, accum 1800...100, accum 1900...100, accum 2000...100, accum 2100...100, accum 2200...100, accum 2300...100, accum 2400...100, accum 2500...100, accum 2600...100, accum 2700...100, accum 2800...100, accum 2900...100, accum 3000...100, accum 3100...100, accum 3200...100, accum 3300...100, accum 3400...100, accum 3500...100, accum 3600...100, accum 3700...100, accum 3800...100, accum 3900...100, accum 4000...100, accum 4100...100, accum 4200...100, accum 4300...100, accum 4400...100, accum 4500...100, accum 4600...100, accum 4700...100, accum 4800...100, accum 4900...100, accum 5000...100, accum 5100...100, accum 5200...100, accum 5300...100, accum 5400...100, accum 5500...100, accum 5600...1

In [43]:
from dafnedset import fun_transformations as ftr

In [44]:
opts = bs.FunBufferOptions(batch_size=30,niter=2)
cfb = fop.read_db(ds.CASES,batch_size=123)
provider = map(ftr.label_batch([1.,0.]),cfb)

buf = bs.FunBuffer(options=opts, providers=[provider])

In [45]:
for i in buf:
    batch = i
    print(f'{len(batch)}',end='...')

30...30...30...30...30...30...30...30...30...30...30...30...30...30...30...30...30...30...30...30...30...30...30...30...30...30...30...30...30...30...30...30...30...30...30...30...30...30...30...30...30...30...30...30...30...30...30...30...30...30...30...30...30...30...30...30...30...30...30...30...30...30...30...30...30...30...30...30...30...30...30...30...30...30...30...30...30...30...30...30...30...30...30...30...30...30...30...30...30...30...30...30...30...30...30...30...30...30...30...30...30...30...30...30...30...30...30...30...30...30...30...30...30...30...30...30...30...30...30...30...30...30...30...30...30...30...30...30...30...30...30...30...30...30...30...30...30...30...30...30...30...30...30...30...30...30...30...30...30...30...30...30...30...30...30...30...30...30...30...30...30...30...30...30...30...30...30...30...30...30...30...30...30...30...30...30...30...30...30...30...30...30...30...30...30...30...30...30...30...30...30...30...30...30...30...30...30...30...30...30...

In [46]:
opts = bs.FunBufferOptions(batch_size=30,niter=2)
cfb = fop.read_db(ds.CASES,batch_size=123)
label_tr = ftr.label_batch([1.,0.])

buf = cfb.map(label_tr)

In [47]:
for i in buf:
    batch = i
    print(f'{len(batch)}',end='...')

123...123...123...123...123...123...123...123...123...123...123...123...123...123...123...123...123...123...123...123...123...123...123...123...123...123...123...123...123...123...123...123...123...123...123...123...123...123...123...123...123...123...123...123...123...123...123...123...123...123...123...123...123...123...123...123...123...123...123...123...123...123...123...123...123...123...123...123...123...123...123...123...123...123...123...123...123...123...123...36...

In [48]:
len(batch.column('etiqueta').tolist())

36

In [49]:
from dafnedset import split_simple as fss

In [50]:
fss = reload(fss)

In [51]:
cfb = fop.read_db(ds.CASES,batch_size=123)
spliter = fss.FunSplitter(cfb,[1.,2.,1.])

In [52]:
for i in spliter:
    print(len(i),[len(j) for j in i],sum([len(j) for j in i]),sep='->',end='...')

3->[31, 62, 30]->123...3->[31, 62, 30]->123...3->[31, 62, 30]->123...3->[31, 62, 30]->123...3->[31, 62, 30]->123...3->[31, 62, 30]->123...3->[31, 62, 30]->123...3->[31, 62, 30]->123...3->[31, 62, 30]->123...3->[31, 62, 30]->123...3->[31, 62, 30]->123...3->[31, 62, 30]->123...3->[31, 62, 30]->123...3->[31, 62, 30]->123...3->[31, 62, 30]->123...3->[31, 62, 30]->123...3->[31, 62, 30]->123...3->[31, 62, 30]->123...3->[31, 62, 30]->123...3->[31, 62, 30]->123...3->[31, 62, 30]->123...3->[31, 62, 30]->123...3->[31, 62, 30]->123...3->[31, 62, 30]->123...3->[31, 62, 30]->123...3->[31, 62, 30]->123...3->[31, 62, 30]->123...3->[31, 62, 30]->123...3->[31, 62, 30]->123...3->[31, 62, 30]->123...3->[31, 62, 30]->123...3->[31, 62, 30]->123...3->[31, 62, 30]->123...3->[31, 62, 30]->123...3->[31, 62, 30]->123...3->[31, 62, 30]->123...3->[31, 62, 30]->123...3->[31, 62, 30]->123...3->[31, 62, 30]->123...3->[31, 62, 30]->123...3->[31, 62, 30]->123...3->[31, 62, 30]->123...3->[31, 62, 30]->123...3->[31, 62,

In [53]:
pt1 = fss.FunPart(spliter,1)

In [54]:
for i in pt1:
    print(len(i),end='...')

62...62...62...62...62...62...62...62...62...62...62...62...62...62...62...62...62...62...62...62...62...62...62...62...62...62...62...62...62...62...62...62...62...62...62...62...62...62...62...62...62...62...62...62...62...62...62...62...62...62...62...62...62...62...62...62...62...62...62...62...62...62...62...62...62...62...62...62...62...62...62...62...62...62...62...62...62...62...62...18...

In [55]:
cfb = fop.read_db(ds.CASES,batch_size=123)
spliter = fss.FunSplitter(cfb,[1.,2.,1.])

In [56]:
pt1 = fss.FunPart(spliter,1)

for i in pt1:
    print(len(i),end='...')

62...62...62...62...62...62...62...62...62...62...62...62...62...62...62...62...62...62...62...62...62...62...62...62...62...62...62...62...62...62...62...62...62...62...62...62...62...62...62...62...62...62...62...62...62...62...62...62...62...62...62...62...62...62...62...62...62...62...62...62...62...62...62...62...62...62...62...62...62...62...62...62...62...62...62...62...62...62...62...18...

In [57]:
len(spliter.cache[0])

80

In [58]:
cfb = fop.read_db(ds.CASES,batch_size=123)
spliter = fss.FunSplitter(cfb,[1.,2.,1.])

In [59]:
for i in spliter[1]:
    print(len(i),end='...')

123...123...123...123...123...123...123...123...123...123...123...123...123...123...123...123...123...123...123...123...123...123...123...123...123...123...123...123...123...123...123...123...123...123...123...123...123...123...123...119...

In [62]:
from dafnedset.presets_simple.pos10 import data

In [63]:
for i in data:
    print(len(i),end='...')

100...100...100...100...100...100...100...100...100...100...100...100...100...100...100...100...100...100...100...100...100...100...100...100...100...100...100...100...100...100...100...100...100...100...100...100...100...100...100...100...100...100...100...100...100...100...100...100...100...100...100...100...100...100...100...100...100...100...100...100...100...100...100...100...100...100...100...100...28...

In [64]:
from dafnedset import base_simple as bs

In [65]:
buf = bs.FunBuffer(options=data.options,providers=data.providers)

In [66]:
buf

In [67]:
#OBS: Los splitters se resetean solos
for i in buf:
    print(len(i),end='...')

100...100...100...100...100...100...100...100...100...100...100...100...100...100...100...100...100...100...100...100...100...100...100...100...100...100...100...100...100...100...100...100...100...100...100...100...100...100...100...100...100...100...100...100...100...100...100...100...100...100...100...100...100...100...100...100...100...100...100...100...100...100...100...100...100...100...100...100...28...